Vancouver housing prices prediction and analysis project

Dataset source: https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
data = pd.read_csv("Housing.csv")

In [36]:
data

,Number,Address,List Date,Price,Days on market,Total floor area,Year Built,Age,Lot Size
0,1,3178 GRAVELEY STREET,5/8/2020,1500000,18,2447,1946,74,5674.00
1,2,1438 E 28TH AVENUE,1/22/2020,1300000,7,2146,1982,38,3631.98
2,3,2831 W 49TH AVENUE,6/18/2019,2650000,1,3108,1929,90,9111.00
3,4,2645 TRIUMPH STREET,6/18/2019,1385000,28,2602,1922,97,4022.70
4,5,741-743 E 10TH AVENUE,11/28/2019,1590000,17,1843,1970,49,4026.00
...,...,...,...,...,...,...,...,...,...
1297,1298,65 W KING EDWARD AVENUE,8/22/2019,2630000,42,3035,1939,80,7456.00
1298,1299,3150 E 52ND AVENUE,8/17/2019,1450000,14,2282,1974,45,3993.00
1299,1300,4478 PRINCE ALBERT STREET,2/24/2020,2798000,4,3501,2016,4,3960.00
1300,1301,4038 MILLER STREET,4/5/2019,900000,194,2440,1912,107,3297.00


In [37]:
data.shape

(1302, 9)

In [38]:
data.dtypes


Number                int64
Address              object
List Date            object
Price                 int64
Days on market        int64
Total floor area      int64
Year Built            int64
Age                   int64
Lot Size            float64
dtype: object

Preparing data

In [39]:
#removing number and year build columns, as they are not crucial for analysis, as well as removing the street number, assuming there is no significant different in prices on houses on the same street
data = data.loc[:,~data.columns.isin(['Number', 'Year Built'])].copy()

street = [];

for i in range(len(data['Address'])):
    street.append(data['Address'][i].split(' ',1)[1])

data['Street'] = street
data.drop(['Address'], axis = 1, inplace=True)
data

,List Date,Price,Days on market,Total floor area,Age,Lot Size,Street
0,5/8/2020,1500000,18,2447,74,5674.00,GRAVELEY STREET
1,1/22/2020,1300000,7,2146,38,3631.98,E 28TH AVENUE
2,6/18/2019,2650000,1,3108,90,9111.00,W 49TH AVENUE
3,6/18/2019,1385000,28,2602,97,4022.70,TRIUMPH STREET
4,11/28/2019,1590000,17,1843,49,4026.00,E 10TH AVENUE
...,...,...,...,...,...,...,...
1297,8/22/2019,2630000,42,3035,80,7456.00,W KING EDWARD AVENUE
1298,8/17/2019,1450000,14,2282,45,3993.00,E 52ND AVENUE
1299,2/24/2020,2798000,4,3501,4,3960.00,PRINCE ALBERT STREET
1300,4/5/2019,900000,194,2440,107,3297.00,MILLER STREET


In [40]:
data

,List Date,Price,Days on market,Total floor area,Age,Lot Size,Street
0,5/8/2020,1500000,18,2447,74,5674.00,GRAVELEY STREET
1,1/22/2020,1300000,7,2146,38,3631.98,E 28TH AVENUE
2,6/18/2019,2650000,1,3108,90,9111.00,W 49TH AVENUE
3,6/18/2019,1385000,28,2602,97,4022.70,TRIUMPH STREET
4,11/28/2019,1590000,17,1843,49,4026.00,E 10TH AVENUE
...,...,...,...,...,...,...,...
1297,8/22/2019,2630000,42,3035,80,7456.00,W KING EDWARD AVENUE
1298,8/17/2019,1450000,14,2282,45,3993.00,E 52ND AVENUE
1299,2/24/2020,2798000,4,3501,4,3960.00,PRINCE ALBERT STREET
1300,4/5/2019,900000,194,2440,107,3297.00,MILLER STREET


In [41]:
#Establish the deal date, remove list date and days on market

data['List Date'] = pd.to_datetime(data['List Date'], yearfirst=True)

dealdate = []

for i in range(len(data['List Date'])):
  dealdate.append(data['List Date'][i] + pd.Timedelta(days=data['Days on market'][i]))

data['Deal Date'] = dealdate

data.drop(['List Date', 'Days on market'], axis = 1, inplace=True)

data['Deal Date'] = data['Deal Date'].dt.to_period('M')

data



,Price,Total floor area,Age,Lot Size,Street,Deal Date
0,1500000,2447,74,5674.00,GRAVELEY STREET,2020-05
1,1300000,2146,38,3631.98,E 28TH AVENUE,2020-01
2,2650000,3108,90,9111.00,W 49TH AVENUE,2019-06
3,1385000,2602,97,4022.70,TRIUMPH STREET,2019-07
4,1590000,1843,49,4026.00,E 10TH AVENUE,2019-12
...,...,...,...,...,...,...
1297,2630000,3035,80,7456.00,W KING EDWARD AVENUE,2019-10
1298,1450000,2282,45,3993.00,E 52ND AVENUE,2019-08
1299,2798000,3501,4,3960.00,PRINCE ALBERT STREET,2020-02
1300,900000,2440,107,3297.00,MILLER STREET,2019-10
